## Preparing Data

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import random
import numpy as np
import datasets
import tqdm

In [ ]:
seed = 1234

random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True

### Dataset

In [ ]:
import sys

sys.path.insert(1, "minbpe-master")

In [ ]:
import pandas as pd

df1 = pd.read_csv("pairs_dataset.csv")
df2 = pd.read_csv("generated_pairs.csv")
df = pd.concat([df1, df2])

In [ ]:
df

In [ ]:
# add special tokens
lines = "<sos>" + df["equation"] + " " + df["answer"] + "<eos>"
str_for_vocab_training = lines.str.cat(sep=" ")

In [ ]:
from minbpe import BasicTokenizer

VOCAB_SIZE = 4096

tokenizer = BasicTokenizer()
# doesn't allow adding special tokens; make it learn <pad>
str_for_vocab_training += "<pad>" * 10
tokenizer.train(str_for_vocab_training, vocab_size=VOCAB_SIZE)

In [ ]:
# eq_lines = "<sos>" + df["equation"] + "<eos>"
eq_lines = df["equation"]
ans_lines = "<sos>" + df["answer"] + "<eos>"
eqs_tokenized = np.array(eq_lines.apply(lambda line: list(tokenizer.encode(line))))
ans_tokenized = np.array(ans_lines.apply(lambda line: list(tokenizer.encode(line))))

In [ ]:
seq_length = max(
    len(max(eqs_tokenized, key=len)), len(max(ans_tokenized, key=len))
)  # max seq len


def pad(lines_tokenized_basic):
    for i in range(len(lines_tokenized_basic)):
        while len(lines_tokenized_basic[i]) < seq_length:
            lines_tokenized_basic[i].append(tokenizer.encode("<pad>")[0])
        lines_tokenized_basic[i] = np.array(lines_tokenized_basic[i])
    lines_tokenized_basic = np.array(lines_tokenized_basic)
    return lines_tokenized_basic


eqs_tokenized = pad(eqs_tokenized)
ans_tokenized = pad(ans_tokenized)

In [ ]:
len(eqs_tokenized)

In [ ]:
from datasets import Dataset, DatasetDict

dataset_dict = {"eqs": eqs_tokenized, "ans": ans_tokenized}
dataset = Dataset.from_dict(dataset_dict)

In [ ]:
train_testvalid = dataset.train_test_split(test_size=0.2)
test_valid = train_testvalid["test"].train_test_split(test_size=0.5)
train_test_valid_dataset = DatasetDict(
    {
        "train": train_testvalid["train"],
        "test": test_valid["test"],
        "valid": test_valid["train"],
    }
)

In [ ]:
pad_index = tokenizer.encode("<pad>")[0]

## Data Loaders

In [ ]:
def get_collate_fn(pad_index):
    def collate_fn(batch):
        batch_eqs = [example["eqs"] for example in batch]
        batch_ans = [example["ans"] for example in batch]
        batch_eqs = nn.utils.rnn.pad_sequence(batch_eqs, padding_value=pad_index)
        batch_ans = nn.utils.rnn.pad_sequence(batch_ans, padding_value=pad_index)
        batch = {
            "eqs": batch_eqs,
            "ans": batch_ans,
        }
        return batch

    return collate_fn

In [ ]:
def get_data_loader(dataset, batch_size, pad_index, shuffle=False):
    collate_fn = get_collate_fn(pad_index)
    data_loader = torch.utils.data.DataLoader(
        dataset=dataset,
        batch_size=batch_size,
        collate_fn=collate_fn,
        shuffle=shuffle,
    )
    return data_loader

In [ ]:
data_type = "torch"
format_columns = ["eqs", "ans"]

train_data = train_test_valid_dataset["train"].with_format(
    type=data_type, columns=format_columns, output_all_columns=True
)

valid_data = train_test_valid_dataset["valid"].with_format(
    type=data_type,
    columns=format_columns,
    output_all_columns=True,
)

test_data = train_test_valid_dataset["test"].with_format(
    type=data_type,
    columns=format_columns,
    output_all_columns=True,
)

In [ ]:
batch_size = 128

train_data_loader = get_data_loader(train_data, batch_size, pad_index, shuffle=True)
valid_data_loader = get_data_loader(valid_data, batch_size, pad_index)
test_data_loader = get_data_loader(test_data, batch_size, pad_index)

## Building the Model

### Encoder

In [ ]:
class Encoder(nn.Module):
    def __init__(
        self,
        rnn_type,
        input_dim,
        embedding_dim,
        hidden_dim,
        n_layers,
        dropout,
        bidirectional=False,
    ):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        self.rnn_type = rnn_type
        if rnn_type == "lstm":
            self.rnn = nn.LSTM(
                embedding_dim,
                hidden_dim,
                n_layers,
                dropout=dropout,
                bidirectional=bidirectional,
            )
        elif rnn_type == "rnn":
            self.rnn = nn.RNN(
                embedding_dim,
                hidden_dim,
                n_layers,
                dropout=dropout,
                bidirectional=bidirectional,
            )
        elif rnn_type == "gru":
            self.rnn = nn.GRU(
                embedding_dim,
                hidden_dim,
                n_layers,
                dropout=dropout,
                bidirectional=bidirectional,
            )
        self.dropout = nn.Dropout(dropout)

    def forward(self, src):
        # src = [src length, batch size]
        embedded = self.dropout(self.embedding(src))
        # embedded = [src length, batch size, embedding dim]
        if self.rnn_type == "lstm":
            outputs, (hidden, cell) = self.rnn(embedded)
            return hidden, cell
        else:
            outputs, hidden = self.rnn(embedded)
            return hidden
        # outputs = [src length, batch size, hidden dim * n directions]
        # hidden = [n layers * n directions, batch size, hidden dim]
        # cell = [n layers * n directions, batch size, hidden dim]
        # outputs are always from the top hidden layer

### Decoder

In [ ]:
class Decoder(nn.Module):
    def __init__(
        self,
        rnn_type,
        output_dim,
        embedding_dim,
        hidden_dim,
        n_layers,
        dropout,
        bidirectional=False,
    ):
        super().__init__()
        self.output_dim = output_dim
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.rnn_type = rnn_type
        self.embedding = nn.Embedding(output_dim, embedding_dim)
        if rnn_type == "lstm":
            self.rnn = nn.LSTM(
                embedding_dim,
                hidden_dim,
                n_layers,
                dropout=dropout,
                bidirectional=bidirectional,
            )
        elif rnn_type == "rnn":
            self.rnn = nn.RNN(
                embedding_dim,
                hidden_dim,
                n_layers,
                dropout=dropout,
                bidirectional=bidirectional,
            )
        elif rnn_type == "gru":
            self.rnn = nn.GRU(
                embedding_dim,
                hidden_dim,
                n_layers,
                dropout=dropout,
                bidirectional=bidirectional,
            )
        if bidirectional:
            self.fc_out = nn.Linear(2 * hidden_dim, output_dim)
        else:
            self.fc_out = nn.Linear(hidden_dim, output_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, input, hidden, cell=None):
        # input = [batch size]
        # hidden = [n layers * n directions, batch size, hidden dim]
        # cell = [n layers * n directions, batch size, hidden dim]
        # n directions in the decoder will both always be 1, therefore:
        # hidden = [n layers, batch size, hidden dim]
        # context = [n layers, batch size, hidden dim]
        input = input.unsqueeze(0)
        # input = [1, batch size]
        embedded = self.dropout(self.embedding(input))
        # embedded = [1, batch size, embedding dim]
        if self.rnn_type == "lstm":
            output, (hidden, cell) = self.rnn(embedded, (hidden, cell))
            prediction = self.fc_out(output.squeeze(0))
            return prediction, hidden, cell
        else:
            output, hidden = self.rnn(embedded, hidden)
            prediction = self.fc_out(output.squeeze(0))
            return prediction, hidden
        # output = [seq length, batch size, hidden dim * n directions]
        # hidden = [n layers * n directions, batch size, hidden dim]
        # cell = [n layers * n directions, batch size, hidden dim]
        # seq length and n directions will always be 1 in this decoder, therefore:
        # output = [1, batch size, hidden dim]
        # hidden = [n layers, batch size, hidden dim]
        # cell = [n layers, batch size, hidden dim]
        # prediction = self.fc_out(output.squeeze(0))
        # prediction = [batch size, output dim]

### Seq2Seq

In [ ]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        assert (
            encoder.hidden_dim == decoder.hidden_dim
        ), "Hidden dimensions of encoder and decoder must be equal!"
        assert (
            encoder.n_layers == decoder.n_layers
        ), "Encoder and decoder must have equal number of layers!"

    def forward(self, src, trg, teacher_forcing_ratio):
        # src = [src length, batch size]
        # trg = [trg length, batch size]
        # teacher_forcing_ratio is probability to use teacher forcing
        # e.g. if teacher_forcing_ratio is 0.75 we use ground-truth inputs 75% of the time
        batch_size = trg.shape[1]
        trg_length = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        # tensor to store decoder outputs
        outputs = torch.zeros(trg_length, batch_size, trg_vocab_size).to(self.device)
        # last hidden state of the encoder is used as the initial hidden state of the decoder
        if self.encoder.rnn_type == "lstm":
            hidden, cell = self.encoder(src)
        else:
            hidden = self.encoder(src)
        # hidden = [n layers * n directions, batch size, hidden dim]
        # cell = [n layers * n directions, batch size, hidden dim]
        # first input to the decoder is the <sos> tokens
        input = trg[0, :]
        # input = [batch size]
        for t in range(1, trg_length):
            # insert input token embedding, previous hidden and previous cell states
            # receive output tensor (predictions) and new hidden and cell states
            if self.encoder.rnn_type == "lstm":
                output, hidden, cell = self.decoder(input, hidden, cell)
            else:
                output, hidden = self.decoder(input, hidden)
            # output = [batch size, output dim]
            # hidden = [n layers, batch size, hidden dim]
            # cell = [n layers, batch size, hidden dim]
            # place predictions in a tensor holding predictions for each token
            outputs[t] = output
            # decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            # get the highest predicted token from our predictions
            top1 = output.argmax(1)
            # if teacher forcing, use actual next token as next input
            # if not, use predicted token
            input = trg[t] if teacher_force else top1
            # input = [batch size]
        return outputs

## Training the Model

### Train functions

In [ ]:
# weight initialization
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [ ]:
# train loop


def train_fn(
    model, data_loader, optimizer, criterion, clip, teacher_forcing_ratio, device
):
    model.train()
    epoch_loss = 0
    for i, batch in enumerate(tqdm.tqdm(data_loader)):
        src = batch["eqs"].to(device)
        trg = batch["ans"].to(device)
        # src = [src length, batch size]
        # trg = [trg length, batch size]
        optimizer.zero_grad()
        output = model(src, trg, teacher_forcing_ratio)
        # output = [trg length, batch size, trg vocab size]
        output_dim = output.shape[-1]
        output = output[1:].view(-1, output_dim)
        # output = [(trg length - 1) * batch size, trg vocab size]
        trg = trg[1:].view(-1)
        # trg = [(trg length - 1) * batch size]
        loss = criterion(output, trg)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        epoch_loss += loss.item()
    return epoch_loss / len(data_loader)

### Eval functions

In [ ]:
# eval loop


def evaluate_fn(model, data_loader, criterion, device):
    model.eval()
    epoch_loss = 0
    with torch.no_grad():
        for i, batch in enumerate(data_loader):
            src = batch["eqs"].to(device)
            trg = batch["ans"].to(device)
            # src = [src length, batch size]
            # trg = [trg length, batch size]
            output = model(src, trg, 0)  # turn off teacher forcing
            # output = [trg length, batch size, trg vocab size]
            output_dim = output.shape[-1]
            output = output[1:].view(-1, output_dim)
            # output = [(trg length - 1) * batch size, trg vocab size]
            trg = trg[1:].view(-1)
            # trg = [(trg length - 1) * batch size]
            loss = criterion(output, trg)
            epoch_loss += loss.item()
    return epoch_loss / len(data_loader)

In [ ]:
valid_eqs = [
    "2xy\mathrm{d}x + (x^2 - y^2)\mathrm{d}y = 0",  # в полных дифференциалах
    "\frac{3x^2 + y^2}{y^2}\mathrm{d}x - \frac{2x^3 + 5y}{y^3}\mathrm{d}y",  # в полных дифференциалах
    "y^{\prime}=\mathrm{tg}{\frac{y}{x}}+{\frac{y}{x}}",  # однородное
    "y^{\prime}=\cos^{2}{\frac{y}{x}}+{\frac{y}{x}}",  # однородное
    "y^{\prime}-y={\frac{e^{x}}{x^{2}}}",  # линейное 1-го порядка
    "(2x+y^{2})y^{\prime}=y",  # линейное 1-го порядка
    "y y^{\prime3}+x=1",  # не разрешенное относительно производной
    "y^{\prime^{3}}+y^{2}=y y^{\prime}(y^{\prime}+1)",  # не разрешенное относительно производной
    "2y^{\prime}-\frac{y}{x}=\frac{4x^{2}}{y}",  # уравнение Бернулли
    "xy^{\prime}-2y={\frac{x}{y}}",  # уравнение Бернулли
    "2y^{\prime\prime}+3y^{\prime}-5y=10",  # неоднородные линейные
    "y^{\prime\prime}-2y^{\prime}-8y=x^{2}+3",  # неоднородные линейные
    "{\frac{x\,d x+y\,d y}{y\,\overline{{{1+x^{2}+y^{2}}}}}}+{\frac{y\,d x-x\,d y}{x^{2}+y^{2}}}=0",  # интегрирующий множитель
    "(x^{2}y^{2}-1)\,d y+2x y^{3}\,d x=0",  # интегрирующий множитель
]
valid_eqs_answers = [
    "3x^2 - y^3 = C",
    "x + \frac{x^3}{y^2} + \frac{5}{y} = C",
    "y=x\arcsin(C x)",
    "y(x)=x\tan^{-1}(c_{1}+\log(x))",
    "y(x)=c_{1}\,e^{x}-{\frac{e^{x}}{x}}",
    "x=y^{2}(\ln y + C)",
    "(x-1)^{4/3}+y^{4/3}=C",
    "4y=(x+C)^{2}",
    "y(x)=-{\sqrt{x}}\;{\sqrt{c_{1}+2x^{2}}}",
    "y(x)={\frac{\sqrt{x}\;{\sqrt{c_{1}\,x^{3}-2}}}{\sqrt{3}}}",
    "y(x)=c_{1}\;e^{-(5x)/2}+c_{2}\;e^{x}-2",
    "y(x)=c_{1}\;e^{-2x}+c_{2}\;e^{4x}-{\frac{x^{2}}{8}}+{\frac{x}{16}}-{\frac{27}{64}}",
    "{\sqrt{{1+x^{2}+y^{2}}}}+\arctan{\frac{x}{y}}=C",
    "x^{2}y+{\frac{1}{y}}=C",
]

In [ ]:
import pickle


def predict_valid_eqs(model, file_name):
    preds = []
    for eq in valid_eqs:
        preds.append(predict(eq, model))
    d = dict(zip(valid_eqs_answers, preds))
    with open(f"valid_eqs_preds/{file_name}.pickle", "wb") as handle:
        pickle.dump(d, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
def detokenize(sentence, is_eq: bool):
    full_str = tokenizer.decode(sentence)
    if not is_eq:
        answer = full_str[5:]
        end_token_idx = answer.find("<eos>")
        answer = answer[:end_token_idx]
    else:
        answer = full_str
    answer = answer.replace("<pad>", "")
    return answer


def tokenize(sentence, is_eq: bool):
    if not is_eq:
        sentence = "<sos>" + sentence + "<eos>"
    res = tokenizer.encode(sentence)
    while len(res) < seq_length:
        res.append(tokenizer.encode("<pad>")[0])
    return res

In [ ]:
def predict(
    sentence,
    model,
    input_is_tokenized=False,
    device="cuda",
    max_output_length=25,
):
    model.eval()
    with torch.no_grad():
        if input_is_tokenized:
            ids = sentence
        else:
            ids = tokenize(sentence, is_eq=True)
        tensor = torch.LongTensor(ids).unsqueeze(-1).to(device)
        if model.encoder.rnn_type == "lstm":
            hidden, cell = model.encoder(tensor)
        else:
            hidden = model.encoder(tensor)
        inputs = tokenizer.encode("<sos>")
        for _ in range(max_output_length):
            inputs_tensor = torch.LongTensor([inputs[-1]]).to(device)
            if model.encoder.rnn_type == "lstm":
                output, hidden, cell = model.decoder(inputs_tensor, hidden, cell)
            else:
                output, hidden = model.decoder(inputs_tensor, hidden)
            predicted_token = output.argmax(-1).item()
            inputs.append(predicted_token)
            if predicted_token == tokenizer.encode("<eos>")[0]:
                break
        tokens = detokenize(inputs, is_eq=False)[4:]
    return tokens

In [ ]:
import evaluate

bleu = evaluate.load("bleu")

val_references = [
    detokenize(sentence.tolist(), is_eq=False) for sentence in valid_data["ans"]
]
test_references = [
    detokenize(sentence.tolist(), is_eq=False) for sentence in test_data["ans"]
]


def bleu_score(preds, refs):
    return bleu.compute(predictions=preds, references=refs)["bleu"]


def accuracy(preds, refs):
    equal_count = 0
    for i in range(len(refs)):
        if preds[i] == refs[i]:
            equal_count += 1
    return equal_count / len(refs)

In [ ]:
# result saving

result = {
    "rnn_type": [],
    "optimizer": [],
    "bidirectional": [],
    "hidden_dim": [],
    "n_layers": [],
    "learning_rate": [],
    "teacher_forcing_ratio": [],
    "epoch": [],
    "val_bleu": [],
    "val_accuracy": [],
    "test_bleu": [],
    "test_accuracy": [],
}


def update_result():
    result["rnn_type"].append(rnn_type)
    result["teacher_forcing_ratio"].append(teacher_forcing_ratio)
    result["optimizer"].append(optimizer_name)
    result["bidirectional"].append(bidirectional)
    result["n_layers"].append(n_layers)
    result["hidden_dim"].append(hidden_dim)
    result["learning_rate"].append(lr)
    result["epoch"].append(epoch + 1)
    result["val_bleu"].append(round(val_bleu, 3))
    result["test_bleu"].append(round(test_bleu, 3))
    result["val_accuracy"].append(round(val_accuracy, 3))
    result["test_accuracy"].append(round(test_accuracy, 3))

    res_df = pd.DataFrame(result)
    res_df.to_csv("results.csv", index=False)

## Train

In [ ]:
input_dim = VOCAB_SIZE
output_dim = VOCAB_SIZE
encoder_embedding_dim = 256
decoder_embedding_dim = 256
encoder_dropout = 0.5
decoder_dropout = 0.5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

rnn_type_options = ["rnn", "gru", "lstm"]
optimizer_options = ["Adam", "AdamW"]
teacher_forcing_ratio_options = [0.1, 0.3, 0.5, 0.7]
lr_options = [0.0001, 0.001, 0.01, 0.1]
n_layers_options = [2, 4, 6, 8]
hidden_dim_options = [256, 512, 1024]
for rnn_type in rnn_type_options:
    for teacher_forcing_ratio in teacher_forcing_ratio_options:
        for optimizer_name in optimizer_options:
            for bidirectional in [False, True]:
                for hidden_dim in hidden_dim_options:
                    for n_layers in n_layers_options:
                        for lr in lr_options:
                            encoder = Encoder(
                                rnn_type,
                                input_dim,
                                encoder_embedding_dim,
                                hidden_dim,
                                n_layers,
                                encoder_dropout,
                                bidirectional,
                            )

                            decoder = Decoder(
                                rnn_type,
                                output_dim,
                                decoder_embedding_dim,
                                hidden_dim,
                                n_layers,
                                decoder_dropout,
                                bidirectional,
                            )

                            model = Seq2Seq(encoder, decoder, device).to(device)
                            model.apply(init_weights)
                            print(
                                f"The model has {count_parameters(model):,} trainable parameters"
                            )

                            if optimizer_name == "Adam":
                                optimizer = optim.Adam(model.parameters(), lr=lr)
                            else:
                                optimizer = optim.AdamW(model.parameters(), lr=lr)
                            criterion = nn.CrossEntropyLoss(ignore_index=pad_index)

                            n_epochs = 6
                            clip = 1.0
                            best_valid_loss = float("inf")
                            for epoch in range(n_epochs):
                                print(f"EPOCH {epoch+1}")
                                unique_name = f"type-{rnn_type}_optim-{optimizer_name}__teacher-forcing-{teacher_forcing_ratio}_bidir-{int(bidirectional)}_hiddim-{hidden_dim}_layers-{n_layers}_lr-{lr}_epoch-{epoch+1}"
                                train_loss = train_fn(
                                    model,
                                    train_data_loader,
                                    optimizer,
                                    criterion,
                                    clip,
                                    teacher_forcing_ratio,
                                    device,
                                )
                                valid_loss = evaluate_fn(
                                    model,
                                    valid_data_loader,
                                    criterion,
                                    device,
                                )
                                if valid_loss < best_valid_loss:
                                    best_valid_loss = valid_loss
                                    torch.save(
                                        model.state_dict(),
                                        f"encoder_decoder_models/model_{unique_name}.pt",
                                    )
                                print(
                                    f"\tTrain Loss: {train_loss:7.3f} | Train PPL: {np.exp(train_loss):7.3f}"
                                )
                                print(
                                    f"\tValid Loss: {valid_loss:7.3f} | Valid PPL: {np.exp(valid_loss):7.3f}"
                                )

                                # compute metrics
                                val_predictions = [
                                    predict(sentence, model, True)
                                    for sentence in valid_data["eqs"]
                                ]
                                test_predictions = [
                                    predict(sentence, model, True)
                                    for sentence in test_data["eqs"]
                                ]
                                val_bleu = bleu_score(
                                    preds=val_predictions, refs=val_references
                                )
                                test_bleu = bleu_score(
                                    preds=test_predictions, refs=test_references
                                )
                                val_accuracy = accuracy(
                                    preds=val_predictions, refs=val_references
                                )
                                test_accuracy = accuracy(
                                    preds=test_predictions, refs=test_references
                                )

                                # save preds on valid_eqs
                                predict_valid_eqs(model, f"pred_{unique_name}")

                                print(
                                    f"\tValid BLEU: {val_bleu:7.3f} | Valid Accuracy: {val_accuracy:7.3f}"
                                )
                                update_result()